# LIBRARIES

In [1]:
import os
import ast
import ccxt
import time
import pickle
import numpy as np
import configparser
import pandas as pd
import google.cloud
from pandas_gbq import gbq
from datetime import datetime
from google.cloud import bigquery
from IPython.display import clear_output

# FUNCTIONS

In [2]:
#FUNCIÓN QUE LEE PICKLE DE VARIABLE
def readPickleVariable(variable_name):    
    return pickle.load(open(variable_name+".pickle","rb"))

#_____

#FUNCIÓN QUE DESCARGA BASE DE DATOS DE MERCADO DE BIGQUERY
def downloadDataBaseBigQuery(rows=None):
    
    #_____GOOGLE CLOUD CONECTION
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/bigQueryAccess.json"

    #_____SI NO SE DA UN NÚMERO ESPECÍFICO DE FILAS POR PARÁMETRO
    if rows==None:
    
        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet_Reports.GENERAL_BOOK]",project_id="dogwood-terra-308100",dialect="legacy")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
        
    else:
        
        #_____DESCARGAR BASE DE DATOS
        candlesDataBase_BigQuery=gbq.read_gbq("SELECT * FROM [dogwood-terra-308100:surfNet_Reports.GENERAL_BOOK] LIMIT "+str(rows),project_id="dogwood-terra-308100",dialect="legacy")
        candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    #_____REESTRUCTURAR BASE DE DATOS
    candlesDataBase_BigQuery.reset_index(inplace=True,drop=True)
    
    #_____RETURN
    return candlesDataBase_BigQuery

# CHECK

# RUN

In [ ]:
#_____GOOGLE CLOUD CONECTION
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

#_____CREAR CONEXIÓN CON BASE DE DATOS EN BIGQUERY
bigquery_client=bigquery.Client(project="dogwood-terra-308100")
dataset_ref=bigquery_client.dataset("surfNet_Reports")
table_ref=dataset_ref.table("GENERAL_BOOK")

#_____WHILE LOOP PARA ITERACIÓN PERMANENTE
while True:
    
    #_____TRY DE CONTINGENCIA
    try:

        #_____DESCARGAR ARCHIVOS PICKLE DE TRANSACCIONES
        tradingBook_MonoBot_1=readPickleVariable("tradingBook_MonoBot_1")
        tradingBook_MonoBot_2=readPickleVariable("tradingBook_MonoBot_2")
        tradingBook_MonoBot_3=readPickleVariable("tradingBook_MonoBot_3")
        tradingBook_MonoBot_4=readPickleVariable("tradingBook_MonoBot_4")

        #_____CREAR BASE DE DATOS
        tradingBook_MonoBot_General=pd.DataFrame()

        #_____MIRAR LONGITUD DE BASES DE DATOS
        if len(tradingBook_MonoBot_1)>0:
            tradingBook_MonoBot_General=tradingBook_MonoBot_General.append(tradingBook_MonoBot_1)
        if len(tradingBook_MonoBot_2)>0:
            tradingBook_MonoBot_General=tradingBook_MonoBot_General.append(tradingBook_MonoBot_2)
        if len(tradingBook_MonoBot_3)>0:
            tradingBook_MonoBot_General=tradingBook_MonoBot_General.append(tradingBook_MonoBot_3)
        if len(tradingBook_MonoBot_4)>0:
            tradingBook_MonoBot_General=tradingBook_MonoBot_General.append(tradingBook_MonoBot_4)

        #_____SI LONGITUD GENERAL ES MAYOER QUE CERO
        if len(tradingBook_MonoBot_General)>0:

            #_____CREAR BASE DE DATOS
            tradingBook_MonoBot_General.reset_index(drop=True,inplace=True)

            #_____DAR FORMATO A COLUMNAS
            tradingBook_MonoBot_General.DATE_BUY=pd.to_datetime(tradingBook_MonoBot_General.DATE_BUY)
            tradingBook_MonoBot_General.DATE_SELL=pd.to_datetime(tradingBook_MonoBot_General.DATE_SELL)
            tradingBook_MonoBot_General.BUY_CRYPTO_AMOUNT=tradingBook_MonoBot_General.BUY_CRYPTO_AMOUNT.astype(float)
            tradingBook_MonoBot_General.NET_BUY_COST=tradingBook_MonoBot_General.NET_BUY_COST.astype(float)
            tradingBook_MonoBot_General.BUY_FEE=tradingBook_MonoBot_General.BUY_FEE.astype(float)
            tradingBook_MonoBot_General.BUY_FEE_PERCET=tradingBook_MonoBot_General.BUY_FEE_PERCET.astype(float)
            tradingBook_MonoBot_General.SELL_CRYPTO_AMOUNT=tradingBook_MonoBot_General.SELL_CRYPTO_AMOUNT.astype(float)
            tradingBook_MonoBot_General.NET_SELL_COST=tradingBook_MonoBot_General.NET_SELL_COST.astype(float)
            tradingBook_MonoBot_General.SELL_FEE=tradingBook_MonoBot_General.SELL_FEE.astype(float)
            tradingBook_MonoBot_General.NET_UTILITY=tradingBook_MonoBot_General.NET_UTILITY.astype(float)

            #_____RENOMBRAR COLUMAS
            tradingBook_MonoBot_General.rename(columns={'NET_UTILITY': 'NET_MARGIN_UTILITY'}, inplace=True)

            #_____CALCULAR VALORES
            tradingBook_MonoBot_General["NET_BUY_PRICE"]=tradingBook_MonoBot_General.NET_BUY_COST/tradingBook_MonoBot_General.BUY_CRYPTO_AMOUNT
            tradingBook_MonoBot_General["NET_SELL_PRICE"]=tradingBook_MonoBot_General.NET_SELL_COST/tradingBook_MonoBot_General.SELL_CRYPTO_AMOUNT
            tradingBook_MonoBot_General["NET_MONEY_UTILITY"]=tradingBook_MonoBot_General.NET_SELL_COST-tradingBook_MonoBot_General.NET_BUY_COST

            #_____DINERO ACUMULADO
            for index in range(0,len(tradingBook_MonoBot_General)):
                if index == 0:
                    tradingBook_MonoBot_General.at[index,"NET_MONEY_UTILITY_ACUM"]=tradingBook_MonoBot_General.at[index,"NET_MONEY_UTILITY"]
                else:
                    tradingBook_MonoBot_General.at[index,"NET_MONEY_UTILITY_ACUM"]=tradingBook_MonoBot_General.at[index-1,"NET_MONEY_UTILITY_ACUM"]+tradingBook_MonoBot_General.at[index,"NET_MONEY_UTILITY"]

            #_____DESCARGAR BASE DE DATOS EXISTENTE
            tradingBook_MonoBot_Cloud = downloadDataBaseBigQuery()

            #_____SACAR LISTA PARA ELIMINACIÓN DE DUPLICADO
            tradingBook_MonoBot_General=tradingBook_MonoBot_General[[x not in list(tradingBook_MonoBot_Cloud.BUY_ID.values) for x in list(tradingBook_MonoBot_General.BUY_ID.values)]]
            tradingBook_MonoBot_General.reset_index(inplace=True,drop=True,level=0)
            tradingBook_MonoBot_General.dropna(subset=["NET_MARGIN_UTILITY"],inplace=True)
            tradingBook_MonoBot_General.reset_index(inplace=True,drop=True,level=0)

            #_____SI TENGO DATOS PARA SUBIR
            if len(tradingBook_MonoBot_General)>0:

                #_____INSERTAR FILAR POR PEDAZOS
                rows_candle_list=list(np.arange(0,len(tradingBook_MonoBot_General),10000))
                for i in range(len(rows_candle_list)):
                    try:
                        bigquery_client.insert_rows(bigquery_client.get_table(table_ref), tradingBook_MonoBot_General[rows_candle_list[i]:rows_candle_list[i+1]].values.tolist())
                    except:
                        bigquery_client.insert_rows(bigquery_client.get_table(table_ref), tradingBook_MonoBot_General[rows_candle_list[i]:].values.tolist())

            #_____LIMPIAR OUTPUT
            clear_output()

        #_____ESPERAR UN TIEMPO PARA NO SOBRECARGAR APIS
        time.sleep(10)
    
    except:
        
        #_____IMPRIMIR ERROR
        print("[[ERROR]]")
        
        #_____ESPERAR UN TIEMPO PARA NO SOBRECARGAR APIS

        time.sleep(10)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# START

In [ ]:
# #_____GOOGLE CLOUD CONECTION
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/home/ubuntu/SurfNet/predictionModels/PREDICTION/bigQueryAccess.json"

# #_____CREAR CONEXIÓN CON BASE DE DATOS EN BIGQUERY
# bigquery_client=bigquery.Client(project="dogwood-terra-308100")
# dataset_ref=bigquery_client.dataset("surfNet_Reports")
# table_ref=dataset_ref.table("GENERAL_BOOK")

# table=dataset_ref.table("GENERAL_BOOK")
# table=bigquery.Table(table)

# #_____DAR FORMATO DE TABLA PARA CARGAR
# table.schema = (
#     bigquery.SchemaField('MACHINE', 'STRING'),
#     bigquery.SchemaField('TRANSACTION_ID', 'STRING'),
#     bigquery.SchemaField('MARKET', 'STRING'),
#     bigquery.SchemaField('BUY_ID', 'STRING'),
#     bigquery.SchemaField('DATE_BUY', 'DATETIME'),
#     bigquery.SchemaField('SELL_ID', 'STRING'),
#     bigquery.SchemaField('DATE_SELL', 'DATETIME'),
#     bigquery.SchemaField('BUY_CRYPTO_AMOUNT', 'FLOAT'),
#     bigquery.SchemaField('NET_BUY_COST', 'FLOAT'),
#     bigquery.SchemaField('BUY_FEE', 'FLOAT'),
#     bigquery.SchemaField('BUY_FEE_PERCET', 'FLOAT'),
#     bigquery.SchemaField('BUY_CURRENCY_FEE', 'STRING'),
#     bigquery.SchemaField('SELL_CRYPTO_AMOUNT', 'FLOAT'),
#     bigquery.SchemaField('NET_SELL_COST', 'FLOAT'),
#     bigquery.SchemaField('SELL_FEE', 'FLOAT'),
#     bigquery.SchemaField('SELL_CURRENCY_FEE', 'STRING'),
#     bigquery.SchemaField('NET_MARGIN_UTILITY', 'FLOAT'),
#     bigquery.SchemaField('NET_BUY_PRICE', 'FLOAT'),
#     bigquery.SchemaField('NET_SELL_PRICE', 'FLOAT'),
#     bigquery.SchemaField('NET_MONEY_UTILITY', 'FLOAT'),
#     bigquery.SchemaField('NET_MONEY_UTILITY_ACUM', 'FLOAT')
# )

# bigquery_client.create_table(table)